# Example 2: Coupled Transmon Qubits

In [1]:
%load_ext autoreload
%autoreload 2
import time

In [2]:
import json
import design as d
import names as n

with open("design_variables.json") as in_file:
    initial_design_variables = json.load(in_file)

from qdesignoptimizer.utils.chip_generation import create_chip_base, ChipType
from qdesignoptimizer.utils.utils import close_ansys


## Design assembly

In [3]:
close_ansys()

In [4]:
CHIP_NAME = "multi_transmon_chip"
OPEN_GUI = True
chip_type = ChipType(size_x="10mm", size_y="10mm", size_z="-300um")
design, gui = create_chip_base(
    chip_name=CHIP_NAME, chip_type=chip_type, open_gui=OPEN_GUI
)

n.add_design_variables_to_design(design, initial_design_variables)

In [5]:
def render_qiskit_metal_design(design, gui):
    d.add_transmon_plus_resonator(design, group=n.NBR_1)
    d.add_transmon_plus_resonator(design, group=n.NBR_2)

    d.add_coupler(design)

    d.add_route_interconnects(design)

    d.add_launch_pads(design)

    d.add_chargeline(design, group=n.NBR_1)
    d.add_chargeline(design, group=n.NBR_2)

    gui.rebuild()
    gui.autoscale()


render_qiskit_metal_design(design, gui)

## MiniStudies

In [6]:
import mini_studies as ms
import optimization_targets as ot
import parameter_targets as pt
import plot_settings as ps

from qdesignoptimizer.design_analysis import DesignAnalysis, DesignAnalysisState
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qdesignoptimizer.design_analysis_types import MeshingMap

# Single resonator

In [7]:
"""
Here we want to study the following permutations:
- fine mesh conductor
- no fine mesh

single run
14 passes
look at convergence of imaginary part
"""

'\nHere we want to study the following permutations:\n- fine mesh conductor\n- no fine mesh\n\nsingle run\n14 passes\nlook at convergence of imaginary part\n'

In [8]:
def CoupledLineTee_mesh_names(comp_names):
    all_names_to_mesh = [
                        f"prime_cpw_{comp_names}", f"second_cpw_{comp_names}", 
                         ]
    return all_names_to_mesh

In [9]:
MINI_STUDY_GROUP = n.NBR_1
MINI_STUDY = ms.get_mini_study_res(group=MINI_STUDY_GROUP)
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

opt_targets = ot.get_opt_targets_2qubits_resonator_coupler(
    groups=[MINI_STUDY_GROUP],
    opt_target_qubit_freq=False,
    opt_target_qubit_anharm=False,
    opt_target_resonator_freq=True,
    opt_target_resonator_kappa=True,
    opt_target_resonator_qubit_chi=False,
)

In [10]:
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path='out/' + CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS,
    meshing_map=[MeshingMap(component_class = CoupledLineTee, mesh_names = CoupledLineTee_mesh_names)]
    
)

group_runs = 1
group_passes = 14
delta_f = 0.001
for i in range(group_runs):
    design_analysis.update_nbr_passes(group_passes)
    design_analysis.update_delta_f(delta_f)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

[INFO|2025-03-09 11:56:45]: self.eig_solver.sim.setup 
{
    "name": "Resonator_setup",
    "reuse_selected_design": true,
    "reuse_setup": true,
    "min_freq_ghz": 1,
    "n_modes": 1,
    "max_delta_f": 0.5,
    "max_passes": 10,
    "min_passes": 1,
    "min_converged": 1,
    "pct_refinement": 30,
    "basis_order": 1,
    "vars": {
        "Lj": "10 nH",
        "Cj": "0 fF"
    }
}
INFO 11:56AM [connect_project]: Connecting to Ansys Desktop API...
INFO 11:56AM [load_ansys_project]: 	Opened Ansys App
INFO 11:56AM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 11:56AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/lukassp/Documents/Ansoft/
	Project:   Project11
INFO 11:56AM [connect_design]: No active design found (or error getting active design).
INFO 11:56AM [connect]: 	 Connected to project "Project11". No design detected
11:56AM 52s WARNING [activate_ansys_design]: The design_name=get_mini_study_res was not in active project.  Designs in active 

pyaedt INFO: using existing logger.
pyaedt INFO: Launching PyAEDT outside AEDT with CPython and PythonNET.
pyaedt INFO: AEDT installation Path C:\Program Files\AnsysEM\AnsysEM21.2\Win64.
pyaedt INFO: Launching AEDT with module PythonNET.
pyaedt INFO: Ansoft.ElectronicsDesktop.2021.2 Started with process ID 30596.
pyaedt INFO: pyaedt v0.6.46
pyaedt INFO: Python version 3.10.15 | packaged by Anaconda, Inc. | (main, Oct  3 2024, 07:22:19) [MSC v.1929 64 bit (AMD64)]
pyaedt INFO: No project is defined. Project Project11 exists and has been read.
pyaedt INFO: Active Design set to get_mini_study_res
pyaedt INFO: Aedt Objects initialized
pyaedt INFO: Variable Manager initialized
pyaedt INFO: Design Loaded
pyaedt INFO: Successfully loaded project materials !
pyaedt INFO: Materials Loaded


INFO 11:56AM [analyze]: Analyzing setup Setup
[WARNING|2025-03-09 12:54:00]: No junctions found, skipping EPR analysis.


Design "get_mini_study_res" info:
	# eigenmodes    1
	# variations    1
Design "get_mini_study_res" info:
	# eigenmodes    1
	# variations    1


12:54PM 00s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  CRITICAL: QEventDispatcherWin32::wakeUp: Failed to post a message (Not enough quota is available to process this command.)

12:54PM 00s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  CRITICAL: OleSetClipboard: Failed to set mime data (application/x-qt-image) on clipboard: COM error 0xffffffff800401d0  (Unknown error 0x0800401d0) (The parameter is incorrect.)



### Optimize capacitance target

In [12]:
MINI_STUDY_BRANCH = 1
MINI_STUDY = ms.get_mini_study_resonator_capacitance(group=MINI_STUDY_BRANCH)
opt_targets = ot.get_opt_target_capacitance(group=MINI_STUDY_BRANCH)

RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)
# %matplotlib inline
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path="out/" + CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS_CAPACITANCE,
)

[INFO|2025-03-10 09:37:53]: self.eig_solver.sim.setup 
{
    "name": "Resonator_setup",
    "reuse_selected_design": true,
    "reuse_setup": true,
    "min_freq_ghz": 1,
    "n_modes": 1,
    "max_delta_f": 0.5,
    "max_passes": 10,
    "min_passes": 1,
    "min_converged": 1,
    "pct_refinement": 30,
    "basis_order": 1,
    "vars": {
        "Lj": "10 nH",
        "Cj": "0 fF"
    }
}
INFO 09:37AM [connect_project]: Connecting to Ansys Desktop API...
INFO 09:37AM [load_ansys_project]: 	Opened Ansys App
INFO 09:37AM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 09:37AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/lukassp/Documents/Ansoft/
	Project:   Project11
INFO 09:37AM [connect_design]: 	Opened active design
	Design:    get_mini_study_res [Solution type: Eigenmode]
INFO 09:37AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 09:37AM [connect]: 	Connected to project "Project11" and design "get_mini_study_res" 😀 

09:

In [13]:
group_runs = 1
group_passes_cap = 10

for i in range(group_runs):
    design_analysis.update_nbr_passes_capacitance_ministudies(group_passes_cap)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

[INFO|2025-03-10 09:37:55]: Updated_design_vars
{
    "cpw_width": "10 um",
    "cpw_gap": "6 um",
    "design_var_width_qubit_1": "400um",
    "design_var_width_qubit_2": "400um",
    "design_var_cl_pos_x_qubit_1": "-2600um",
    "design_var_cl_pos_y_qubit_1": "-1800um",
    "design_var_cl_pos_x_qubit_2": "-2600um",
    "design_var_cl_pos_y_qubit_2": "1800um",
    "design_var_length_resonator_1_capacitance": "20um",
    "design_var_length_resonator_2_capacitance": "20um",
    "design_var_lj_qubit_1": "12.1nH",
    "design_var_lj_qubit_2": "9.1nH",
    "design_var_cj_qubit_1": "0fF",
    "design_var_cj_qubit_2": "0fF",
    "design_var_length_resonator_1": "7500um",
    "design_var_length_resonator_2": "6500um",
    "design_var_length_coupler_1to2": "4000um",
    "design_var_coupl_length_qubit_1_resonator_1": "100um",
    "design_var_coupl_length_qubit_2_resonator_2": "100um",
    "design_var_coupl_length_resonator_1_tee": "170um",
    "design_var_coupl_length_resonator_2_tee": "160um"


## View Optimization results

In [ ]:
design_analysis.get_cross_kerr_matrix(iteration=-1)

In [11]:
design_analysis.get_eigenmode_results()

,,Freq. (GHz),Quality Factor,Kappas (kHz),Freq. (Hz),Kappas (Hz)
variation,mode,,,,,
0,0,7.684739,1.155793e+06,6.64889,7.684739e+09,6648.8898


In [ ]:
close_ansys()

# Analytical formula for coupling Q

In [ ]:
import numpy as np

In [ ]:
def qc(imp, cap, omega):
    return np.pi / (imp**2 * omega**2 * cap**2)

In [ ]:
freq= 7.683e9
freq/qc(50, 1.47947e-15, 2*np.pi*freq)/1e3

In [ ]:
1.479-1.253

In [ ]:
0.226/1.253
